---
jupyter: python3
---

# Sympy: Laplace Transform {#sec-sympy-Laplace-transform}

## Importing sympy

In [ ]:
import sympy
from sympy import laplace_transform, inverse_laplace_transform, pi
sympy.init_printing()

from IPython.display import display

print("sympy: ", sympy.__version__)

* [_laplace_trabsform_expansion.py](_laplace_transform_expansion.py)

  * `laplace_tranform_()` is the expanded version of `laplace_transform()`

  * `subs_()` is the modified function of `sub()` method


In [ ]:
#| echo: false
# Process Systems Lab., SeoulTech
# April 15, 2021 Authored by Kee-Youn Yoo
# April 4, 2024 Modifed for sympy version 1.12

import sympy
from sympy import integrate, laplace_transform
from functools import reduce

def subs_(e, s1, s2):

    if isinstance(e, sympy.LaplaceTransform):
        s_ = e.args[2]
        return e.subs(s_, s).subs(s1, s2).subs(s, s_)

    if isinstance(e, (sympy.Add, sympy.Mul, 
                      sympy.Derivative, sympy.Integral, sympy.Subs)):
        tp = type(e)      
        return tp(*[subs_(arg, s1, s2) for arg in e.args])

    return e


def laplace_transform_(*e, **f):
    
    t_ = e[1]
    
    if isinstance(e[0], (int, float)):
        return laplace_transform(*e, **f)[0]

    k = len(e[0].args)
    
    terms = []
    for i in range(k):
        if  k == 1:
            terms.append(e[0])
        else:
            if isinstance(e[0], (sympy.Mul, sympy.Derivative, sympy.Integral)):
                terms.append(e[0])
                break
            else:
                terms.append(e[0].args[i])
    
    m = len(terms)
    if m == 0:
        return laplace_transform(*e, **f)[0]
    
    Leq = sympy.Float('0')
    for i in range(m):

        flag = 0
        l = len(terms[i].args) 
        if l == 1:
            terms__ = terms[i]
        else:
            terms__ = sympy.Integer('1')
            for j in range(l):
                if isinstance(terms[i], (sympy.Derivative, sympy.Integral)):
                    terms__ = terms[i]
                    break
                else: 
                    if isinstance(terms[i].args[j], sympy.exp):
                        a = terms[i].args[j].args[0].args
                        if len(a) == 2:
                            flag = a[0]
                        else:
                            flag = a[0] *a[2]                                                         
                    else:
                        terms__ *= terms[i].args[j]

        Leq_ = laplace_transform_expansion(laplace_transform(terms__, e[1], e[2], **f)[0])

        if flag != 0: 
            Leq_ = Leq_.subs(e[2], e[2] -flag)

        Leq += Leq_

    return Leq.doit()


def laplace_transform_expansion(e):
    """
    Evaluate the laplace transforms of derivatives, integrals, and composites of functions
    """       
    
    if isinstance(e, sympy.LaplaceTransform):
        
        ex, t, s = e.args
        
        # Preliminaries --------------------------

        if len(ex.args) == 1: 
           
            c = []
            for arg in ex.args[0].args:
                if arg != t: c.append(arg)
                    
            if len(c) == 0:
                return e
            else:
                d = reduce(lambda x, y: x *y, c)
                #return (sympy.LaplaceTransform(ex.subs(d *t, t), t, s/d) /d)
                return (sympy.LaplaceTransform(ex.subs(d *t, t), t, s))
               
        if isinstance(ex.args[0], sympy.Pow): 
            ex = sympy.simplify(ex)
            
        ex0 = ex.args[0]           
        if not isinstance(ex, sympy.Integral):
            ex1 = reduce(lambda x, y: x *y, ex.args[1:])
           
        # -----------------------------------------            
      
        if isinstance(ex, sympy.Derivative):

            n = ex1.args[1]           
            return ((s**n) *sympy.LaplaceTransform(ex0, t, s)
                    -sum([s**(n -i) *sympy.diff(ex0, t, i -1).subs(t, 0) for i in range(1, n +1)]))
        
        elif isinstance(ex, sympy.Integral):        
            
            if len(ex.args[1]) == 3:

                tau, t0, t = ex.args[-1]
                if t0 != 0: return e                
                       
                if len(ex0.args) == 2:
               
                    f, g = ex0.args[0], ex0.args[1]
                
                    if f.args[0] == tau and g.args[0] == t -tau:           
                        return (sympy.LaplaceTransform(f, tau, s).subs(tau, t) 
                               *sympy.LaplaceTransform(g, t -tau, s)).subs(t -tau, t)
                    elif f.args[0] == t -tau and g.args[0] == tau:
                        return (sympy.LaplaceTransform(f, t -tau, s).subs(t -tau, t) 
                               *sympy.LaplaceTransform(g, tau, s)).subs(tau, t)
                    else:
                        return e
                    
                else:
                    n = len(ex.args) -2
                    if n > 0:
                        for i in range(n):
                            tau_, t0_, t_ = ex.args[i +1]
                            ex0 = integrate(ex0, (tau_, 0, t_))
                        ex0 = ex0.subs(tau_, t).subs(t_, t)
                    else:
                        ex0 = ex0.subs(tau, t)
                        
                    return (laplace_transform_expansion(sympy.LaplaceTransform(ex0, t, s)) /s)
            else:
                return e

        elif isinstance(ex0, sympy.exp):         

            c = []
            for arg in ex0.args[0].args:
                if arg != t: c.append(arg)

            d = reduce(lambda x, y: x *y, c)
                       
            if ex0 == sympy.exp(d *t):
                return (laplace_transform_expansion(sympy.LaplaceTransform(ex1, t, s)).subs(s, s -d))
            else:
                return e                
        
        elif isinstance(ex0, sympy.Pow) or ex0 == t:  
            
            if ex0 == t:
                n = 1
            else:
                n = ex0.args[1]
                if not n.is_integer: return e
                      
            c = laplace_transform_expansion(sympy.LaplaceTransform(ex1, t, s))
            
            if isinstance(c, sympy.Add):
                for i in range(len(c.args)):
                    for j in range(len(c.args[i].args)):
                        if isinstance(c.args[i].args[j], sympy.LaplaceTransform):
                            d = c.args[i].args[j].args[-1]
            elif isinstance(c, sympy.Mul):
                for i in range(len(c.args)):
                    if isinstance(c.args[i], sympy.LaplaceTransform):
                        d = c.args[i].args[-1]                   
            elif isinstance(c, sympy.LaplaceTransform):
                d = c.args[-1]
            # else:
            #     return ((-1)**n *sympy.diff(c, (s, n)))
            
            #return ((-1)**n *sympy.diff(c.subs(d, s), (s, n)).subs(s, d))
            s_ = sympy.Symbol('s')

            return (-1)**n *sympy.diff(c.subs(d, s_), (s_, n)).subs(s_, d)

        elif isinstance(ex0, (sympy.Derivative, sympy.Integral)):
                       
            if isinstance(ex1, sympy.exp):
                
                c = []
                for arg in ex1.args[0].args:
                    if arg != t: c.append(arg)

                d = reduce(lambda x, y: x *y, c)

                return (laplace_transform_expansion(sympy.LaplaceTransform(ex0, t, s).subs(s, s -d)))       
            
        elif isinstance(ex0, sympy.Heaviside):          
            
            t, m_a = ex0.args[0].args
            
            if ex1.args[0] == t +m_a:
                f = ex1.subs(t +m_a, t)
                return (sympy.exp(m_a *s) *sympy.LaplaceTransform(f, t, s))
            elif ex1.args[0] == t:
                f = ex1.subs(t, t -m_a)
                return (sympy.exp(m_a *s) *sympy.LaplaceTransform(f, t, s))
            else:
                return e
        
    if isinstance(e, (sympy.Add, sympy.Mul, 
       sympy.Derivative, sympy.Integral, sympy.Subs)):
        tp = type(e)      
        return tp(*[laplace_transform_expansion(arg) for arg in e.args])

    return e

## Symbols and Functions

In [ ]:
a, b = sympy.symbols("a, b", positive=True, constant=True)
n, m = sympy.symbols("n, m", positive=True, constant=True, integer=True)

t, tau, tau1, tau2 = sympy.symbols("t, tau, tau1, tau2", positive=True)

y = sympy.Function("y")
f = sympy.Function("f")
g = sympy.Function("g")

s = sympy.symbols("s")

Y = sympy.Function("Y")
F = sympy.Function("F")
G = sympy.Function("G")

Ly = laplace_transform_(y(t), t, s)
Lf = laplace_transform_(f(t), t, s)
Lg = laplace_transform_(g(t), t, s)

## Laplace transform

In [ ]:
eq = 1
Leq = laplace_transform_(eq, t, s)
Leq

In [ ]:
eq = t
Leq = laplace_transform_(eq, t, s)
Leq

In [ ]:
eq = sympy.exp(-3 *t)
Leq = laplace_transform_(eq, t, s)
Leq

In [ ]:
eq = 2 *sympy.Heaviside(t -3)
Leq = laplace_transform_(eq, t, s)
Leq

In [ ]:
eq = sympy.sin(2 *t)**2
Leq = laplace_transform_(eq, t, s)
Leq

## Inverse Laplace transform

In [ ]:
Leq = 1 / s**3
eq = inverse_laplace_transform(Leq, s, t)
eq

In [ ]:
Leq = (-2 *s +6) / (s**2 + 4)
eq = inverse_laplace_transform(Leq, s, t)
eq

## Laplace transform of derivatives

In [ ]:
eq = y(t).diff(t, 2) +2 *y(t).diff(t) +10 *y(t)
eq

In [ ]:
Leq = subs_(laplace_transform_(eq, t, s), Ly, Y(s))
Leq

In [ ]:
ics = {y(0): 1, y(t).diff(t).subs(t, 0): 1}

Leq = Leq.subs(ics)
Leq

In [ ]:
sol = sympy.solve(Leq, Y(s))[0]
sol

## Laplace transform of integrals

In [ ]:
eq = sympy.integrate(g(tau1), (tau1, 0, t)) +sympy.integrate(sympy.integrate(f(tau2), (tau2, 0, tau1)), (tau1, 0, t))
eq

In [ ]:
Leq = subs_(laplace_transform_(eq, t, s), Lf, F(s))
Leq = subs_(Leq, Lg, G(s))
Leq

## Convolution

In [ ]:
eq = sympy.integrate(f(tau)*g(t -tau), (tau, 0, t)) +sympy.integrate(f(t -tau)*g(tau), (tau, 0, t))
eq

In [ ]:
Leq = subs_(laplace_transform_(eq, t, s), Lf, F(s))
Leq = subs_(Leq, Lg, G(s))
Leq

## First translation theorem

In [ ]:
eq = y(t) *sympy.exp(-a *t)
eq

In [ ]:
Leq = subs_(laplace_transform_(eq, t, s), Ly, Y(s))
Leq

## Second translation theorem

In [ ]:
eq = f(t -a) *sympy.Heaviside(t -a)
eq

In [ ]:
Leq = subs_(laplace_transform_(eq, t, s), Lf, F(s))
Leq

In [ ]:
eq = g(t) *sympy.Heaviside(t -a)
eq

In [ ]:
Leq = laplace_transform_(eq, t, s)
Leq

In [ ]:
eq = sympy.cos(t) *sympy.Heaviside(t -pi)
eq

In [ ]:
Leq = laplace_transform_(eq, t, s)
Leq

## Derivatives of Transforms

In [ ]:
eq = 3 *f(t) *t**m *t**n
eq

In [ ]:
Leq = subs_(laplace_transform_(eq, t, s), Lf, F(s))
Leq

In [ ]:
eq =  t**n *t**m *f(t)*sympy.exp(-2*t)
eq

In [ ]:
Leq = subs_(laplace_transform_(eq, t, s), Lf, F(s))
Leq

---

In [ ]:
eq = t**n *sympy.diff(f(t), t, t)
eq

In [ ]:
Leq = subs_(laplace_transform_(eq, t, s), Lf, F(s))
Leq

In [ ]:
eq = t *sympy.integrate(f(tau), (tau, 0, t))
eq

In [ ]:
Leq = subs_(laplace_transform_(eq, t, s), Lf, F(s))
Leq.doit()

---

In [ ]:
eq = t *f(t) *sympy.exp(-t)
eq

In [ ]:
Leq = subs_(laplace_transform_(eq, t, s), Lf, F(s))
Leq

---

In [ ]:
eq = t *sympy.diff(f(t), t) *sympy.exp(-4*t)
eq

In [ ]:
Leq = subs_(laplace_transform_(eq, t, s), Lf, F(s))
Leq

## Various Transform Examples

In [ ]:
eq = sympy.exp(-a*t) *sympy.diff(f(t), t, t)
eq

In [ ]:
Leq = subs_(laplace_transform_(eq, t, s), Lf, F(s))
Leq

---

In [ ]:
eq = sympy.exp(-4*t) *sympy.integrate(f(tau), (tau, 0, t))
eq

In [ ]:
Leq = subs_(laplace_transform_(eq, t, s), Lf, F(s))
Leq

---

In [ ]:
eq = f(3*a*t)
eq

In [ ]:
Leq = subs_(laplace_transform_(eq, t, s), Lf, F(s))
Leq

$~$

## **Example 1**

A technique that can be used to solve certain ODE problems is to Laplace transform the ODE, which for many problems results in an algebraic equation that is easier to solve. The solution to the algebraic equation can then be transformed back to the original domain with an inverse Laplace transform, to obtain the solution to the original problem. For example, consider the following differential equation for a driven harmonic oscillator:

$$ \frac{d^2 y}{dt^2} + 2\frac{dy}{dt} +10 y = 2\sin 3t $$

$~$

In [ ]:
t = sympy.symbols('t', positive=True)
y = sympy.Function('y')

s = sympy.symbols('s', real=True)
Y = sympy.Function('Y')

Ly = laplace_transform_(y(t), t, s) 

In [ ]:
ode = y(t).diff(t, 2) +2 *y(t).diff(t) +10 *y(t) -2 *sympy.sin(3*t)
ode

In [ ]:
Lode = subs_(laplace_transform_(ode, t, s), Ly, Y(s))
Lode

* At this point, we need to specify the initial conditions for the ODE problem. Here we use 
$y(0)=1$ and $y'(0)=0$, and after creating dictionary that contains these initial conditions, we use it to substitute the values into the Laplace-transformed ODE equation:

In [ ]:
ics = {y(0): 1, y(t).diff(t).subs(t, 0): 0}
Lode = Lode.subs(ics)
Lode

* This is an algebraic equation that can be solved for $Y(s)$

In [ ]:
Ysol = sympy.solve(Lode, Y(s))
Ysol[0]

* The result is a list of solutions, which in this case contains only one element. Performing the inverse Laplace transformation on this expression gives the solution to the original problem in the time domain:

In [ ]:
Yp = sympy.apart(Ysol[0])
Yp

In [ ]:
ysol = sympy.inverse_laplace_transform(Yp.args[0], s, t) +sympy.inverse_laplace_transform(Yp.args[1], s, t)
ysol

## **Example 2**

In some instances, the Laplace transform can be used to solve linear differential equations with variable monomial coefficients 

$$ty'' + y' +ty =0, \;\;y(0)=1, \;y'(0)=0$$

$~$

In [ ]:
s, t = sympy.symbols('s, t', positive=True)

y = sympy.Function('y')
Y  = sympy.Function('Y')
Ly = laplace_transform_(y(t), t, s)

ode = t *y(t).diff(t, 2) +y(t).diff(t) +t *y(t)
ode

$~$

* Reduce the given differential equation to a linear first-order DE in the transformed function $Y(s)=\mathcal{L}_t\left[y(t)\right]$

In [ ]:
ics={y(0): 1, y(t).diff(t).subs(t, 0): 0}

Lode = subs_(laplace_transform_(ode, t, s).subs(ics), Ly, Y(s))
Lode.doit().collect(Y(s).diff(s))

* Solve the first-order ODE for $Y(s)$ and then find $y(t)=\mathcal{L}_t^{-1} \left[Y(s) \right]$

In [ ]:
sol = sympy.dsolve(Lode, Y(s), hint='separable')
sol

In [ ]:
y = inverse_laplace_transform(sol.rhs, s, t)
y

In [ ]:
c = sympy.Eq(y.subs(t, 0), 1)
c

In [ ]:
y = y.subs(c.lhs, c.rhs)
y